In [55]:
import pandas as pd
import numpy as np
from datetime import datetime
import backtrader as bt
import matplotlib.pyplot as plt
import calendar
import os
plt.rcParams["font.sans-serif"]=["SimHei"] #设置画图时的中文显示
plt.rcParams["axes.unicode_minus"]=False #设置画图时的负号显示

import warnings
warnings.filterwarnings('ignore')

In [56]:
code_df=pd.read_csv('code_df_eq.csv')
code_df['year_month'] = pd.to_datetime(code_df['year_month'], format='%b-%y')
code_df = code_df.sort_values('year_month')
code_df['year_month']=code_df['year_month'].dt.to_period('M').dt.to_timestamp(freq='M')
code_df = code_df.set_index('year_month')
# code_df.reset_index(drop=True,inplace=True)
for col in code_df.columns:
    code_df[col] = code_df[col].astype(str)
code_df

,code1,code2,code3,code4,code5
year_month,,,,,
2014-06-30,510410,512010,159928,510170,510230
2014-07-31,510230,159930,510170,510410,159936
2014-08-31,510170,159936,510410,510230,512070
2014-09-30,512010,159928,512070,159930,510410
2014-10-31,510230,512070,510170,159928,510410
...,...,...,...,...,...
2022-10-31,512010,159736,159755,159647,512200
2022-11-30,560800,512480,159997,510170,516570
2022-12-31,510170,159709,159786,516960,159730


In [57]:
weight_df=pd.read_csv('weight_df_eq.csv')
weight_df['year_month'] = pd.to_datetime(weight_df['year_month'], format='%b-%y')
weight_df = weight_df.sort_values('year_month')
weight_df['year_month']=weight_df['year_month'].dt.to_period('M').dt.to_timestamp(freq='M')
weight_df = weight_df.set_index('year_month')
weight_df

,weight1,weight2,weight3,weight4,weight5,weight6,weight7,weight8,weight9
year_month,,,,,,,,,
2014-06-30,1.905649e-01,4.320095e-02,0.086772,0.173639,0.105823,0.077102,0.15,0.072898,0.1
2014-07-31,1.506693e-01,8.992013e-02,0.105106,0.140236,0.114069,0.074973,0.15,0.075027,0.1
2014-08-31,2.602012e-01,1.870000e-17,0.006319,0.215926,0.117554,0.075129,0.15,0.074871,0.1
2014-09-30,2.289610e-01,1.440000e-17,0.059054,0.208207,0.103778,0.073254,0.15,0.076746,0.1
2014-10-31,5.270000e-21,1.372517e-01,0.059260,0.000000,0.003488,0.298384,0.10,0.301616,0.1
...,...,...,...,...,...,...,...,...,...
2022-10-31,1.150000e-21,1.348162e-01,0.064785,0.000000,0.000399,0.299998,0.10,0.300002,0.1
2022-11-30,0.000000e+00,1.345644e-01,0.065356,0.000000,0.000079,0.300914,0.10,0.299086,0.1
2022-12-31,2.002253e-01,2.011361e-02,0.070966,0.200403,0.108292,0.073739,0.15,0.076261,0.1


In [58]:
#加载91个etf交易数据
# Load all ETF data files into a dictionary of DataFrames
dataframes = {}
for filename in os.listdir('etf_new'):
    if filename.endswith('.xls'):
        symbol = filename.split('.')[0]  # Extract symbol from filename
        filepath = os.path.join('etf_new', filename)
        df = pd.read_excel(filepath, sheet_name='Sheet1')
        # #修改数据格式
        df = df.rename(columns={'日期':'date','开盘价(元)':'open','最高价(元)':'high','最低价(元)':'low','收盘价(元)':'close','成交量(份)':'volume'})
        df['date']  = pd.to_datetime(df['date'], unit='d', origin=datetime(1899, 12, 30))
        # df=df.set_index('date')
        df['openinterest']=0
        #对df数据进行一个整合
        df=df[['date','open','high','low','close','volume','openinterest']]
        # print(filename)
        df['volume'] = df['volume'].replace('--', 0)
        # for col in df.columns:
        #     df[col] = df[col].astype(np.float64)
        dataframes[symbol] = df
        # break
    

In [59]:
dataframes['159613']

,date,open,high,low,close,volume,openinterest
0,2022-01-27,0.966,0.966,0.922,0.922,128368829,0
1,2022-01-28,0.922,0.936,0.915,0.920,19707469,0
2,2022-02-07,0.930,0.934,0.910,0.912,10770597,0
3,2022-02-08,0.911,0.920,0.900,0.920,28318754,0
4,2022-02-09,0.921,0.944,0.920,0.943,48438778,0
...,...,...,...,...,...,...,...
297,2023-04-21,1.042,1.042,0.988,0.991,16437300,0
298,2023-04-24,0.990,0.999,0.977,0.984,14483100,0
299,2023-04-25,0.983,0.994,0.965,0.981,17971302,0
300,2023-04-26,0.978,0.980,0.930,0.941,14217300,0


## 通过上证指数校正日期,所有数据统一日期范围，并填充空值！

In [60]:
import tushare as ts
def Get_benchmark_kdata(code, start_date, end_date, index=False):
    df = ts.get_k_data(code, autype='qfq', start=start_date, end=end_date, index=index)
    df['date'] = pd.to_datetime(df['date'])
    df['openinterest'] = 0
    return df[['date', 'open', 'high', 'low', 'close', 'volume',  'openinterest']]

In [61]:
start_date = '2014-01-27'
end_date = '2023-04-31'
benchmark='000001'
dataframes['benchmark'] = Get_benchmark_kdata(benchmark, start_date, end_date,True)

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2


In [62]:
dataframes['benchmark']

,date,open,high,low,close,volume,openinterest
17,2014-01-27,2044.27,2044.85,2029.63,2033.30,88815426.0,0
18,2014-01-28,2036.40,2047.13,2026.99,2038.51,72529043.0,0
19,2014-01-29,2042.18,2051.58,2039.77,2049.91,73865462.0,0
20,2014-01-30,2045.93,2045.93,2031.47,2033.08,62615184.0,0
21,2014-02-07,2022.32,2044.73,2014.70,2044.50,73598755.0,0
...,...,...,...,...,...,...,...
2265,2023-04-24,3300.81,3303.09,3264.93,3275.41,354255626.0,0
2266,2023-04-25,3276.40,3288.57,3229.45,3264.87,380895729.0,0
2267,2023-04-26,3255.22,3277.98,3239.99,3264.10,357917842.0,0
2268,2023-04-27,3256.23,3299.30,3253.11,3285.88,341590534.0,0


In [64]:
def Checkdata(kdata):
    for secu in set(kdata.keys()) - set(['benchmark']):
        secu_kdata = kdata['benchmark'][['date']].merge(kdata[secu], how='left') #用大盘的日期来补充每一支股票的日期
        # 获取每个月的最后一个交易日
        last_days = secu_kdata.groupby(secu_kdata['date'].dt.to_period('M'))['date'].transform('last').dt.day
        
        # 将结果转换为0和1的标记
        secu_kdata['is_last_day'] = (secu_kdata['date'].dt.day == last_days).astype(int)
        secu_kdata['end_day'] = secu_kdata.groupby(secu_kdata['date'].dt.to_period('M'))['date'].transform('last')
        secu_kdata.set_index(['date'], inplace=True)  # 设date为index
        secu_kdata.fillna(method='ffill', inplace=True)  # 上市后的数据用前日数据进行补充
        secu_kdata.fillna(value=0, inplace=True)  # 上市前的数据用0补充
        secu_kdata['signal']=0
        secu_kdata['weight'] =0
        kdata[secu] = secu_kdata
    kdata.pop('benchmark')
    
    return kdata

In [65]:
dataframes2 = dataframes.copy()
dataframes2 = Checkdata(dataframes2)


In [66]:
#检验是否都一样的date
for symbol, df in dataframes2.items():
    if df.shape[0]!=2253:
        print(symbol)

In [67]:
dataframes2

{'159613':              open   high    low  close      volume  openinterest  is_last_day  \
 date                                                                            
 2014-01-27  0.000  0.000  0.000  0.000         0.0           0.0            0   
 2014-01-28  0.000  0.000  0.000  0.000         0.0           0.0            0   
 2014-01-29  0.000  0.000  0.000  0.000         0.0           0.0            0   
 2014-01-30  0.000  0.000  0.000  0.000         0.0           0.0            1   
 2014-02-07  0.000  0.000  0.000  0.000         0.0           0.0            0   
 ...           ...    ...    ...    ...         ...           ...          ...   
 2023-04-24  0.990  0.999  0.977  0.984  14483100.0           0.0            0   
 2023-04-25  0.983  0.994  0.965  0.981  17971302.0           0.0            0   
 2023-04-26  0.978  0.980  0.930  0.941  14217300.0           0.0            0   
 2023-04-27  0.931  0.944  0.910  0.918  13151902.0           0.0            0   
 2023-

In [68]:
dataframes2 = dataframes.copy()
dataframes2 = Checkdata(dataframes2)

date_list = list(code_df.index)
for day in date_list:
    codes_list = code_df.loc[day]
    for i in range(len(codes_list)):
        code = codes_list[i]
        df = dataframes2[code].copy()
        weight = weight_df.loc[day][i]
        # print(code,weight)
        df.loc[(df['end_day'] == day) & (df['is_last_day'] == 1),'signal'] = 1
        df.loc[(df['end_day'] == day) & (df['is_last_day'] == 1),'weight'] = weight
        dataframes2[code] = df   

In [83]:
df = dataframes2['gold']
df[df['signal']==1]

,open,high,low,close,volume,openinterest,is_last_day,end_day,signal,weight
date,,,,,,,,,,
2014-06-30,0,0,0,261.86,10000000.0,0,1,2014-06-30,1,0.15
2014-07-31,0,0,0,257.79,10000000.0,0,1,2014-07-31,1,0.15
2014-09-30,0,0,0,241.24,10000000.0,0,1,2014-09-30,1,0.15
2014-10-31,0,0,0,230.44,10000000.0,0,1,2014-10-31,1,0.10
2014-12-31,0,0,0,240.59,10000000.0,0,1,2014-12-31,1,0.05
...,...,...,...,...,...,...,...,...,...,...
2022-09-30,0,0,0,390.00,10000000.0,0,1,2022-09-30,1,0.05
2022-10-31,0,0,0,391.76,10000000.0,0,1,2022-10-31,1,0.10
2022-11-30,0,0,0,405.76,10000000.0,0,1,2022-11-30,1,0.10


In [70]:
import pickle
file1 = open('dataframes2.pkl', 'wb')
pickle.dump(dataframes2, file1)
file1.close()


In [71]:
import pickle
file2 = open('dataframes2.pkl', 'rb')
dataframes2 = pickle.load(file2)


## 其他数据

In [72]:
dictA ={}
#加载2个债券数据
bond1=pd.read_excel('general_asset\\CBA00601.CS.xls')
bond1 = bond1.rename(columns={'日期':'date','收盘价':'close'})
bond1['date']  = pd.to_datetime(bond1['date'])
# bond1=bond1.set_index('date')
bond1['volume']=1e7
bond1['openinterest']=0
bond1['open']=0
bond1['high']=0
bond1['low']=0
bond1['close'] = bond1['close'] * 1.4
 #对df数据进行一个整合
bond1 = bond1[['date','open','high','low','close','volume','openinterest']]
dictA['bond1'] = bond1


bond2=pd.read_excel('general_asset\\H11008.CSI.xls')
bond2 = bond2.rename(columns={'日期':'date','收盘价':'close'})
bond2['date']  = pd.to_datetime(bond2['date'])
# bond2 = bond2.set_index('date')
bond2['volume']=1e7
bond2['openinterest']=0
bond2['open']=0
bond2['high']=0
bond2['low']=0
bond2['close'] = bond2['close'] * 1.4
 #对df数据进行一个整合
bond2 = bond2[['date','open','high','low','close','volume','openinterest']]
dictA['bond2'] = bond2

#加载一个海外资产数据
overseas=pd.read_excel('general_asset\\oversees513500.SH.xls')
overseas = overseas.rename(columns={'日期':'date','开盘价(元)':'open','最高价(元)':'high','最低价(元)':'low','收盘价':'close'})
overseas['date']  = pd.to_datetime(overseas['date'])
# overseas=overseas.set_index('date')
overseas['volume']=1e7
overseas['openinterest']=0
 #对df数据进行一个整合
overseas=overseas[['date','open','high','low','close','volume','openinterest']]
dictA['oversees513500.SH'] = overseas


#加载一个商品数据
gold=pd.read_excel('general_asset\\gold1.xls')
gold = gold.rename(columns={'日期':'date','收盘价':'close'})
gold['date']  = pd.to_datetime(gold['date'])
# gold=gold.set_index('date')
gold['volume']=1e7
gold['openinterest']=0
gold['open']=0
gold['high']=0
gold['low']=0
 #对df数据进行一个整合
gold=gold[['date','open','high','low','close','volume','openinterest']]
dictA['gold'] = gold


In [73]:
dictA['gold']

,date,open,high,low,close,volume,openinterest
0,2002-10-30,0,0,0,83.52,10000000.0,0
1,2002-10-31,0,0,0,83.91,10000000.0,0
2,2002-11-01,0,0,0,84.51,10000000.0,0
3,2002-11-04,0,0,0,84.99,10000000.0,0
4,2002-11-05,0,0,0,85.13,10000000.0,0
...,...,...,...,...,...,...,...
4987,2023-04-24,0,0,0,441.64,10000000.0,0
4988,2023-04-25,0,0,0,445.44,10000000.0,0
4989,2023-04-26,0,0,0,445.93,10000000.0,0
4990,2023-04-27,0,0,0,446.72,10000000.0,0


In [74]:
dictA['benchmark'] = Get_benchmark_kdata(benchmark, start_date, end_date,True)

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2


In [75]:
dictB = dictA.copy()
dictB = Checkdata(dictB)

In [76]:
dictB

{'bond1':             open  high  low      close      volume  openinterest  is_last_day  \
 date                                                                            
 2014-01-27     0     0    0  197.66572  10000000.0             0            0   
 2014-01-28     0     0    0  197.61840  10000000.0             0            0   
 2014-01-29     0     0    0  197.60748  10000000.0             0            0   
 2014-01-30     0     0    0  197.61434  10000000.0             0            1   
 2014-02-07     0     0    0  198.15726  10000000.0             0            0   
 ...          ...   ...  ...        ...         ...           ...          ...   
 2023-04-24     0     0    0  302.74538  10000000.0             0            0   
 2023-04-25     0     0    0  302.81034  10000000.0             0            0   
 2023-04-26     0     0    0  302.93270  10000000.0             0            0   
 2023-04-27     0     0    0  303.31476  10000000.0             0            0   
 2023-0

In [77]:
dictB.keys()

dict_keys(['bond1', 'bond2', 'oversees513500.SH', 'gold'])

In [78]:
dictB = dictA.copy()
dictB = Checkdata(dictB)

date_list = list(code_df.index)
for day in date_list:
    for code in dictB.keys():
        # print(code)
        df = dictB[code].copy()
        if code =='bond1':
            weight = weight_df.loc[day,'weight6']
        if code =='bond2':
            weight = weight_df.loc[day,'weight8']
        if code =='gold':
            weight = weight_df.loc[day,'weight7']
        if code =='oversees513500.SH':
            weight = weight_df.loc[day,'weight9']
        df.loc[(df['end_day'] == day) & (df['is_last_day'] == 1),'signal'] = 1
        df.loc[(df['end_day'] == day) & (df['is_last_day'] == 1),'weight'] = weight
        dictB[code]= df

In [79]:
df = dictB['bond1']
df[df['signal']==1]

,open,high,low,close,volume,openinterest,is_last_day,end_day,signal,weight
date,,,,,,,,,,
2014-06-30,0,0,0,206.97222,10000000.0,0,1,2014-06-30,1,0.077102
2014-07-31,0,0,0,205.30832,10000000.0,0,1,2014-07-31,1,0.074973
2014-09-30,0,0,0,209.33654,10000000.0,0,1,2014-09-30,1,0.073254
2014-10-31,0,0,0,213.77062,10000000.0,0,1,2014-10-31,1,0.298384
2014-12-31,0,0,0,216.70754,10000000.0,0,1,2014-12-31,1,0.024578
...,...,...,...,...,...,...,...,...,...,...
2022-09-30,0,0,0,298.49890,10000000.0,0,1,2022-09-30,1,0.025373
2022-10-31,0,0,0,301.24822,10000000.0,0,1,2022-10-31,1,0.299998
2022-11-30,0,0,0,297.49090,10000000.0,0,1,2022-11-30,1,0.300914


In [80]:
dataframes2

{'159613':              open   high    low  close      volume  openinterest  is_last_day  \
 date                                                                            
 2014-01-27  0.000  0.000  0.000  0.000         0.0           0.0            0   
 2014-01-28  0.000  0.000  0.000  0.000         0.0           0.0            0   
 2014-01-29  0.000  0.000  0.000  0.000         0.0           0.0            0   
 2014-01-30  0.000  0.000  0.000  0.000         0.0           0.0            1   
 2014-02-07  0.000  0.000  0.000  0.000         0.0           0.0            0   
 ...           ...    ...    ...    ...         ...           ...          ...   
 2023-04-24  0.990  0.999  0.977  0.984  14483100.0           0.0            0   
 2023-04-25  0.983  0.994  0.965  0.981  17971302.0           0.0            0   
 2023-04-26  0.978  0.980  0.930  0.941  14217300.0           0.0            0   
 2023-04-27  0.931  0.944  0.910  0.918  13151902.0           0.0            0   
 2023-

In [81]:
for key, value in dictB.items():
    print(key)
    dataframes2[key] = value  


bond1
bond2
oversees513500.SH
gold


In [82]:
import pickle
file1 = open('dataframes3.pkl', 'wb')
pickle.dump(dataframes2, file1)
file1.close()